In [22]:
from network import POFLNetWork
from federated_learning.client import Worker
from federated_learning.model import LeNet_Small_Quant

In [20]:
network = POFLNetWork(num_clients=5,
                          global_rounds=10,
                          local_rounds=10,
                          frac_malicous=0.0,
                          dataset='cifar10',
                          model='lenet')
# network.run()

Files already downloaded and verified
Files already downloaded and verified
Block 0 stored.


In [23]:
network.workers = []
X_test, y_test = network.X_test[:1000], network.y_test[:1000] # public test set
for i in range(network.num_clients):
    if i <= network.num_malicous:
        worker = Worker(index=i+1,
                        X_train=network.X_train[i],
                        y_train=network.y_train[i],
                        X_test=None,
                        y_test=None,
                        model=LeNet_Small_Quant(),
                        malicious=True)
    else:
        worker = Worker(index=i+1,
                        X_train=network.X_train[i],
                        y_train=network.y_train[i],
                        X_test=None,
                        y_test=None,
                        model=LeNet_Small_Quant(),
                        malicious=False)
    network.workers.append(worker)

In [31]:
# global_params = network.blockchain.last_block.global_params
# for w in network.workers:
#     w.model.set_params(global_params)
# network.local_train(B=64)
# print("Local training done")
for worker in network.workers:
    # update = worker.local_update(x=X_test, y=y_test, eval=True)
    # worker.send_tx(update, network.blockchain.transaction_pool) 
    # print(f'transaction {worker.index}', network.blockchain.transaction_pool[-1])
    err, acc = worker.evaluate(worker.model, X_test, y_test, B=64)
    print(f'Worker {worker.index} accuracy: {acc}')
# print("Transactions sent")

Worker 1 accuracy: 0.15625
Worker 2 accuracy: 0.20253906305879354
Worker 3 accuracy: 0.1875
Worker 4 accuracy: 0.17734375037252903
Worker 5 accuracy: 0.1474609375


In [46]:
for worker in network.workers:
    err, acc = worker.evaluate(worker.model, X_test, y_test, B=64)
    update = worker.local_update(x=X_test, y=y_test, eval=True)
    print(update['accuracy'], acc)
    # worker.send_tx(update, network.blockchain.transaction_pool) 
    # print(f'transaction {worker.index}', network.blockchain.transaction_pool[-1])

0.203125 0.20253906305879354
0.203125 0.20253906305879354
0.203125 0.20253906305879354
0.203125 0.20253906305879354
0.203125 0.20253906305879354


In [45]:
tx = network.blockchain.transaction_pool[1]
for worker in network.workers:
    worker.set_params(tx.params)
    _, acc = worker.evaluate(worker.model, X_test, y_test, B=64)
    print(f'Worker {worker.index} accuracy: {acc}')

Worker 1 accuracy: 0.20253906305879354
Worker 2 accuracy: 0.20253906305879354
Worker 3 accuracy: 0.20253906305879354
Worker 4 accuracy: 0.20253906305879354
Worker 5 accuracy: 0.20253906305879354


In [43]:
tx.params

[array([[[[-0.03798791, -0.0067296 , -0.02841286, -0.05184913,
            0.09442692],
          [ 0.02474842,  0.08114428, -0.0161978 ,  0.0049566 ,
            0.07829659],
          [-0.09125473, -0.08722431,  0.07531812,  0.02452374,
           -0.06793854],
          [-0.0323753 ,  0.03156762,  0.09514654, -0.10916291,
            0.09975212],
          [ 0.03143502,  0.06201895,  0.0932229 ,  0.06698627,
           -0.08276717]],
 
         [[-0.01667197,  0.09694858, -0.02019564,  0.04240632,
           -0.02972217],
          [-0.10046832,  0.00810761, -0.08627918, -0.02245006,
           -0.06720866],
          [ 0.02198279,  0.00756421, -0.05557569, -0.10539674,
            0.06303845],
          [-0.03950252,  0.10166351, -0.06215288, -0.05046511,
            0.06466633],
          [-0.11548382,  0.02606718,  0.0229425 , -0.04869205,
           -0.05829714]],
 
         [[ 0.05677876, -0.03104515, -0.11915024, -0.07969244,
            0.03381525],
          [-0.03101379,  0